In [7]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib


In [2]:
# Read in data
mod_data = pd.read_csv('appointments_clean.csv')

In [3]:
# Split into X and y
X = mod_data.drop(
    ['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay', 
     'Neighbourhood', 'No-show', 'NoShow'], 
    axis = 1
)
y = mod_data.NoShow

# Test/train split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 314
)

In [ ]:
# Scale data
# scaler = MinMaxScaler().fit(X_train)

# X_train = scaler.transform(X_train)
# X_test  = scaler.transform(X_test)

In [4]:
# Grid search - gradient boosted
param_gb = {
    'loss': ['deviance'],
    'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    'min_samples_split': np.linspace(0.1, 0.5, 9),
    'min_samples_leaf': np.linspace(0.1, 0.5, 9),
    'max_depth': [3, 5, 8],
    'max_features': ['log2', 'sqrt'],
    'criterion': ['friedman_mse'],
    'subsample': np.linspace(0.5, 1.0, 5),
    'n_estimators': [10]
} 

grid_gb01 = RandomizedSearchCV(
    GradientBoostingClassifier(),
    param_gb,
    n_iter = 50,
    refit = True,
    verbose = 3
)

grid_gb02 = RandomizedSearchCV(
    GradientBoostingClassifier(),
    param_gb,
    n_iter = 50,
    scoring = 'brier_score_loss',
    refit = True,
    verbose = 3
)

# Grid search - neural net
param_mlp = {
    'solver': ['lbfgs', 'sgd'], 
    'max_iter': np.arange(10, 21) * 100, 
    'alpha': 10.0 ** -np.arange(1, 10), 
    'hidden_layer_sizes': np.arange(10, 16)
}

grid_mlp01 = RandomizedSearchCV(
    MLPClassifier(),
    param_mlp,
    n_iter = 50,
    refit = True,
    verbose = 3
)

grid_mlp02 = RandomizedSearchCV(
    MLPClassifier(),
    param_mlp,
    n_iter = 50,
    scoring = 'brier_score_loss',
    refit = True,
    verbose = 3
)


In [5]:
gb01 = grid_gb01.fit(X_train, y_train)
gb02 = grid_gb02.fit(X_train, y_train)
mlp01 = grid_mlp01.fit(X_train, y_train)
mlp02 = grid_mlp02.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] subsample=0.625, n_estimators=10, min_samples_split=0.25, min_samples_leaf=0.45000000000000007, max_features=sqrt, max_depth=5, loss=deviance, learning_rate=0.05, criterion=friedman_mse 
[CV]  subsample=0.625, n_estimators=10, min_samples_split=0.25, min_samples_leaf=0.45000000000000007, max_features=sqrt, max_depth=5, loss=deviance, learning_rate=0.05, criterion=friedman_mse, score=0.7991857309034509, total=   0.1s
[CV] subsample=0.625, n_estimators=10, min_samples_split=0.25, min_samples_leaf=0.45000000000000007, max_features=sqrt, max_depth=5, loss=deviance, learning_rate=0.05, criterion=friedman_mse 
[CV]  subsample=0.625, n_estimators=10, min_samples_split=0.25, min_samples_leaf=0.45000000000000007, max_features=sqrt, max_depth=5, loss=deviance, learning_rate=0.05, criterion=friedman_mse, score=0.7991779440846872, total=   0.1s
[CV] subsample=0.625, n_estimators=10, min_samples_split=0.25, min_samples_leaf=0.450000

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV]  subsample=1.0, n_estimators=10, min_samples_split=0.35, min_samples_leaf=0.1, max_features=log2, max_depth=3, loss=deviance, learning_rate=0.15, criterion=friedman_mse, score=0.7991857309034509, total=   0.1s
[CV] subsample=1.0, n_estimators=10, min_samples_split=0.35, min_samples_leaf=0.1, max_features=log2, max_depth=3, loss=deviance, learning_rate=0.15, criterion=friedman_mse 
[CV]  subsample=1.0, n_estimators=10, min_samples_split=0.35, min_samples_leaf=0.1, max_features=log2, max_depth=3, loss=deviance, learning_rate=0.15, criterion=friedman_mse, score=0.8712629415642328, total=   0.1s
[CV] subsample=1.0, n_estimators=10, min_samples_split=0.35, min_samples_leaf=0.1, max_features=log2, max_depth=3, loss=deviance, learning_rate=0.15, criterion=friedman_mse 
[CV]  subsample=1.0, n_estimators=10, min_samples_split=0.35, min_samples_leaf=0.1, max_features=log2, max_depth=3, loss=deviance, learning_rate=0.15, criterion=friedman_mse, score=0.8855325914149443, total=   0.1s
[CV] su

[CV]  subsample=1.0, n_estimators=10, min_samples_split=0.35, min_samples_leaf=0.45000000000000007, max_features=sqrt, max_depth=3, loss=deviance, learning_rate=0.075, criterion=friedman_mse, score=0.7991779440846872, total=   0.1s
[CV] subsample=0.75, n_estimators=10, min_samples_split=0.2, min_samples_leaf=0.5, max_features=sqrt, max_depth=5, loss=deviance, learning_rate=0.01, criterion=friedman_mse 
[CV]  subsample=0.75, n_estimators=10, min_samples_split=0.2, min_samples_leaf=0.5, max_features=sqrt, max_depth=5, loss=deviance, learning_rate=0.01, criterion=friedman_mse, score=0.7991857309034509, total=   0.1s
[CV] subsample=0.75, n_estimators=10, min_samples_split=0.2, min_samples_leaf=0.5, max_features=sqrt, max_depth=5, loss=deviance, learning_rate=0.01, criterion=friedman_mse 
[CV]  subsample=0.75, n_estimators=10, min_samples_split=0.2, min_samples_leaf=0.5, max_features=sqrt, max_depth=5, loss=deviance, learning_rate=0.01, criterion=friedman_mse, score=0.7991779440846872, tota

...

[CV]  solver=sgd, max_iter=1100, hidden_layer_sizes=12, alpha=1e-09, score=-0.05402153067152908, total=   8.9s
[CV] solver=lbfgs, max_iter=1200, hidden_layer_sizes=11, alpha=0.01 ..
[CV]  solver=lbfgs, max_iter=1200, hidden_layer_sizes=11, alpha=0.01, score=-0.05530435909500032, total=   6.3s
[CV] solver=lbfgs, max_iter=1200, hidden_layer_sizes=11, alpha=0.01 ..
[CV]  solver=lbfgs, max_iter=1200, hidden_layer_sizes=11, alpha=0.01, score=-0.054491066429599395, total=  12.3s
[CV] solver=lbfgs, max_iter=1200, hidden_layer_sizes=11, alpha=0.01 ..
[CV]  solver=lbfgs, max_iter=1200, hidden_layer_sizes=11, alpha=0.01, score=-0.05346217549751388, total=  11.9s
[CV] solver=lbfgs, max_iter=1300, hidden_layer_sizes=15, alpha=1e-08 .
[CV]  solver=lbfgs, max_iter=1300, hidden_layer_sizes=15, alpha=1e-08, score=-0.05524458295681886, total=  15.1s
[CV] solver=lbfgs, max_iter=1300, hidden_layer_sizes=15, alpha=1e-08 .
[CV]  solver=lbfgs, max_iter=1300, hidden_layer_sizes=15, alpha=1e-08, score=-0.0542

[CV]  solver=lbfgs, max_iter=1500, hidden_layer_sizes=12, alpha=0.0001, score=-0.05304406244191305, total=   6.9s
[CV] solver=sgd, max_iter=2000, hidden_layer_sizes=13, alpha=1e-09 ...
[CV]  solver=sgd, max_iter=2000, hidden_layer_sizes=13, alpha=1e-09, score=-0.05619803688527284, total=   5.9s
[CV] solver=sgd, max_iter=2000, hidden_layer_sizes=13, alpha=1e-09 ...
[CV]  solver=sgd, max_iter=2000, hidden_layer_sizes=13, alpha=1e-09, score=-0.054967549716609, total=  10.8s
[CV] solver=sgd, max_iter=2000, hidden_layer_sizes=13, alpha=1e-09 ...
[CV]  solver=sgd, max_iter=2000, hidden_layer_sizes=13, alpha=1e-09, score=-0.05740878078824191, total=  10.5s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=0.01 ....
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=0.01, score=-0.05582496237687487, total=  11.0s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=0.01 ....
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=0.01, score=-0.0550105417956

[CV]  solver=lbfgs, max_iter=1900, hidden_layer_sizes=12, alpha=0.01, score=-0.0530268927538936, total=  12.8s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=11, alpha=0.001 ...
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=11, alpha=0.001, score=-0.05779717645887701, total=   9.7s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=11, alpha=0.001 ...
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=11, alpha=0.001, score=-0.05530958878749332, total=   8.4s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=11, alpha=0.001 ...
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=11, alpha=0.001, score=-0.054370418932833876, total=   6.8s
[CV] solver=sgd, max_iter=1500, hidden_layer_sizes=11, alpha=1e-08 ...
[CV]  solver=sgd, max_iter=1500, hidden_layer_sizes=11, alpha=1e-08, score=-0.05656882763021751, total=   5.9s
[CV] solver=sgd, max_iter=1500, hidden_layer_sizes=11, alpha=1e-08 ...
[CV]  solver=sgd, max_iter=1500, hidden_layer_sizes=11, alpha=1e-08, score=-0.05529056262

[CV]  solver=lbfgs, max_iter=1300, hidden_layer_sizes=10, alpha=1e-08, score=-0.05299866753201608, total=  12.5s
[CV] solver=lbfgs, max_iter=1000, hidden_layer_sizes=13, alpha=0.1 ...
[CV]  solver=lbfgs, max_iter=1000, hidden_layer_sizes=13, alpha=0.1, score=-0.05515846632543606, total=  11.8s
[CV] solver=lbfgs, max_iter=1000, hidden_layer_sizes=13, alpha=0.1 ...
[CV]  solver=lbfgs, max_iter=1000, hidden_layer_sizes=13, alpha=0.1, score=-0.05433036886628205, total=   6.5s
[CV] solver=lbfgs, max_iter=1000, hidden_layer_sizes=13, alpha=0.1 ...
[CV]  solver=lbfgs, max_iter=1000, hidden_layer_sizes=13, alpha=0.1, score=-0.053157516612721244, total=  11.5s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=12, alpha=1e-07 ...
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=12, alpha=1e-07, score=-0.055668230664897955, total=   7.1s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=12, alpha=1e-07 ...
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=12, alpha=1e-07, score=-0.05584453

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 25.0min finished


In [8]:
joblib.dump(gb01, 'gb03.sav')
joblib.dump(gb02, 'gb04.sav')
joblib.dump(mlp01, 'mlp03.sav')
joblib.dump(mlp02, 'mlp04.sav')

['mlp04.sav']